# Ćwiczenie 3

Celem ćwiczenia jest imlementacja metody [Minimax z obcinaniem alpha-beta](https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning) do gry Connect Four (czwórki).

W trakcie ćwiczenia można skorzystać z reposytorium z implementacją gry [Connect Four udostępnionym przez Jakuba Łyskawę](https://github.com/lychanl/two-player-games). Ewentualnie, można zaimplementować samemu grę Connect Four (ale, tak aby rozwiązanie miało ten sam interfejs co podany poniżej).

Implementację Minimax należy przetestować używając różną głębokość przeszukiwania. Implementacja Solvera musi zapewniać interfejs jak poniżej, ale można dodać dowolne metody prywatne oraz klasy wspomagające (jeżeli będą potrzebne).

Punktacja:
- Działająca metoda Minimax - **2 pkt**
- Działająca metoda Minimax z obcinaniem alpha-beta - **1.5 pkt**
- Analiza jakości solvera w zależności od głębokości przeszukiwania **1.5pkt**
    - należy zaimplementować w tym celu prostą wizualizację rozgrywki dwóch agentów, bądź kilka przykładów 'z ręki'
- Jakość kodu **2pkt**

Aby importowanie elementów z poniższej komórki działało należy umieścić tego notebooka w tym samym folderze co paczkę `two_player_games`:
```
├── LICENSE
├── README.md
├── minimax.ipynb # <<< HERE
├── test
│   ├── __init__.py
│   ├── test_connect_four.py
│   ├── test_dots_and_boxes.py
│   └── test_pick.py
└── two_player_games
    ├── __init__.py
    ├── games
    │   ├── connect_four.py
    │   └── dots_and_boxes.py
    ├── move.py
    ├── player.py
    └── state.py
```

In [10]:
from typing import Tuple, List
from copy import deepcopy
import random
    
from two_player_games.player import Player
from two_player_games.games.connect_four import (
    ConnectFour,
    ConnectFourMove,
    ConnectFourState
)

In [3]:
ROW_COUNT = 6
COLUMN_COUNT = 7

In [4]:

POINTS_MAP = {
    "player_4": 10000,
    "opponent_4": -1000,
    "player_2": 1,
    "opponent_2": -1,
    "player_3": 10,
    "opponent_3": -100,
}
DEPTH = 1
WINDOW_LEN = 4


In [7]:
class MinMaxSolver:

    def __init__(self, game: ConnectFour):
        self.game = game

    def evaluate_position(self, player: Player)->float:
        points = 0

        # vertical
        windows_in_col = ROW_COUNT - 3
        for col in range(COLUMN_COUNT):
            for i in range(windows_in_col):
                window = [self.game.state.fields[col][i + k] for k in range(WINDOW_LEN)]
                points += self.evaluate_window(window, player)
        # horizontal
        windows_count_row = COLUMN_COUNT - 3
        for row in range(ROW_COUNT):
            for i in range(windows_count_row):
                window = [self.game.state.fields[i + k][row] for k in range(WINDOW_LEN)]
                points += self.evaluate_window(window, player)

        # diagonal
        for row in range(ROW_COUNT - 3):
            for col in range(COLUMN_COUNT - 3):
                window = [
                    self.game.state.fields[col + i][row + i] for i in range(WINDOW_LEN)
                ]
                points += self.evaluate_window(window, player)
                window = [
                    self.game.state.fields[col + 3 - i][row + i]
                    for i in range(WINDOW_LEN)
                ]
                points += self.evaluate_window(window, player)

        return points

    def evaluate_window(self, window: List, player):
        player_count = window.count(player)
        other_player_count = len([el for el in window if (el != player and el != None)])
        none_count = window.count(None)

        if player_count == 4:
            return POINTS_MAP["player_4"]
        if other_player_count == 4:
            return POINTS_MAP["opponent_4"]
        if player_count == 2 and none_count == 2:
            return POINTS_MAP["player_2"]
        if other_player_count == 2 and none_count == 2:
            return POINTS_MAP["opponent_2"]
        if player_count == 3 and none_count == 1:
            return POINTS_MAP["player_3"]
        if other_player_count == 3 and none_count == 1:
            return POINTS_MAP["opponent_3"]
        return 0


    def get_best_move(self)->int:
        best_score = -float("inf")
        best_move = None
        for col in range(COLUMN_COUNT):
            if not self.is_valid_move(col):
                continue
            current_state = deepcopy(self.game.state)
            self.game.make_move(ConnectFourMove(col))
            _, new_score = self.minimax(self.depth, -float("inf"), float("inf"), False)
            self.game.state = current_state
            if new_score > best_score:
                best_move = col
                best_score = new_score
        return best_move  

    def is_valid_move(self, col_index:int)->bool:
        return self.game.state.fields[col_index][ROW_COUNT - 1] == None

    
    def minimax(self, depth, alpha:float, beta:float, is_maximizing_player:bool)-> Tuple[int, float]:
        """Returns column index and score"""

        if depth == 0 or self.game.state.is_finished():
            score = self.evaluate_position(self.game.state.get_current_player())
            return (None, score)

        if is_maximizing_player:
            value = -float("inf")
            best_move = None
            for col in range(COLUMN_COUNT):
                if not self.is_valid_move(col):
                    continue
                current_state = deepcopy(self.game.state)
                self.game.make_move(ConnectFourMove(col))
                _, new_score = self.minimax(depth - 1, alpha, beta, False)
                self.game.state = current_state
                if value < new_score:
                    best_move = col
                    value = new_score
                alpha = max(alpha, value)
                if alpha >= beta:
                    break
            return (best_move, value)
        else:
            value = float("inf")
            best_move = None
            for col in range(COLUMN_COUNT):
                if not self.is_valid_move(col):
                    continue
                current_state = deepcopy(self.game.state)
                self.game.make_move(ConnectFourMove(col))
                _, new_score = self.minimax(depth - 1, alpha, beta, True)
                self.game.state = current_state
                if value > new_score:
                    best_move = col
                    value = new_score
                beta = min(beta, value)
                if beta <= alpha:
                    break
            return (best_move, value)


## Helper functions

In [11]:


def play(depth):
    """ Playing from console with  minimax """
    p1 = Player("a")
    p2 = Player("b")
    game = ConnectFour(
        size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2
    )
    minimax = MinMaxSolver(game)
    minimax.depth = depth
    print(game)
    while not game.state.is_finished():
        move = int(input("Enter move: "))
        game.make_move(ConnectFourMove(move))
        print(game)

        if game.state.is_finished():
            print("Game is finished")
            print("Winner: " + game.state.get_winner().char)
            break

        best_move = minimax.get_best_move()
        game.make_move(ConnectFourMove(best_move))
        print(game)

        if game.state.is_finished():
            print("Winner: " + game.state.get_winner().char)
            break


def minimax_with_random_player(depth, print_game=False):
    p1 = Player("a")
    p2 = Player("b")
    if print_game:
        print("AI player: a")
    game = ConnectFour(
        size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2
    )
    minimax = MinMaxSolver(game)
    minimax.depth = depth

    while not game.state.is_finished():
        for player in ["ai", "random"]:
            if player == "ai":
                best_move = minimax.get_best_move()
                game.make_move(ConnectFourMove(best_move))

            else:
                move = random.randint(0, COLUMN_COUNT - 1)
                game.make_move(ConnectFourMove(move))
            if print_game:
                print(game)
            if game.state.is_finished():
                return game.state.get_winner()


def check_finish(game, winners):
    if game.state.is_finished():
        winner = game.state.get_winner()
        if not winner:
            winners.append(None)
        else:
            winners.append(winner.char)
        return True
    return False


def one_game_demo(depth):
    """ One game between minimax and random player """
    print(f"DEPTH = {depth}")
    winner = minimax_with_random_player(depth=depth, print_game=True)
    if winner:
        print(f"winner: {winner.char}")
    else:
        print("no winner")


def series_of_games_demo(depth_list, no_iterations):
    """ Run series of gaes between random player and minimax"""
    print(f"Games number : {no_iterations}")
    for depth in depth_list:
        print(f"depth: {depth}")
        winners = []
        for _ in range(no_iterations):
            winner = minimax_with_random_player(depth=depth)
            if not winner:
                winners.append(None)
            else:
                winners.append(winner.char)
        random_win_count, ai_win_count = (winners.count("b"), winners.count("a"))
        print(f"random win count: {random_win_count}")
        print(f"ai win count : {ai_win_count}")







## Rozgrywka

### Seria gier dla różnych głębokości

In [12]:
depths = [1, 2, 3]
no_iterations = 100
series_of_games_demo(depths, no_iterations)


Games number : 100
depth: 1
random win count: 11
ai win count : 89
depth: 2
random win count: 26
ai win count : 74
depth: 3
random win count: 6
ai win count : 93


In [18]:
depths = [4, 5]
no_iterations = 50
series_of_games_demo(depths, no_iterations)

Games number : 50
depth: 4
random win count: 9
ai win count : 41
depth: 5
random win count: 0
ai win count : 49


#### Wyniki:

| Depth      | minimax vs random player|
| ----------- | ----------- |
| 1        | 89 %       |
| 2          | 74 %        |
| 3         | 93 %        |
| 4         | 82 %        |
| 5         | 98 %        |


### Przykładowa rozgrywka

In [14]:
depth = 4
one_game_demo(depth)

DEPTH = 4
AI player: a
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[a][ ][ ][ ][ ][ ][ ]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[a][ ][ ][ ][ ][b][ ]
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[a][ ][ ][ ][ ][ ][ ]
[a][ ][ ][ ][ ][b][ ]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[b][ ][ ][ ][ ][ ][ ]
[a][ ][ ][ ][ ][ ][ ]
[a][ ][ ][ ][ ][b][ ]
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[a][ ][ ][ ][ ][ ][ ]
[b][ ][ ][ ][ ][ ][ ]
[a][ ][ ][ ][ ][ ][ ]
[a][ ][ ][ ][ ][b][ ]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[a][ ][ ][ ][ ][ ][ ]
[b][ ][ ][ ][ ][ ][ ]
[a][ ][ ][ ][ ][ ][ ]
[a][b][ ][ ][ ][b][ ]
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[a][ ][ ][ ][ ][ ][ ]
[a][ ][ ][ ][ ]

### Gra z użytkownikiem

In [ ]:
depth = 4
play(depth)

# Wnioski

### Parametry heurystyki 
Duże znaczenie na działanie algorytmu ma dobranie heurystki oceny stanu gry. Kiepska heurystyka powoduje, że algorytm nie potrafi popawnie ocenić stanu gry. Ocena stanu gry pozwala algorytmowi minimax wybrać najlepszy ruch. 

### Głębokość

Skuteczność algorytmu minimax w głównej mierze zależy od głębokości przeszukiwania. Głębokość znacznie zwieksza jednak czas wykonywania obliczeń. Im większa głębokość tym lepsze wyniki algorytmu.

### Alpha-beta pruning
Ucinanie gałęzi za pomocą parametrów alpha i beta znacznie przyspiesza wykonywanie algorytmu.